In [1]:
import numpy as np 
import pandas as pd
import os
import psycopg2
import geopandas as gpd

import importlib
from Secrets import secrets
# importlib.reload(secrets)

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Choose comparable properties

In [14]:
engine = psycopg2.connect(database="postgres", user=secrets.user(), password=secrets.password(), host=secrets.host(), port='5432')
cur = engine.cursor()
cur.execute("""
SELECT 
    p.brn, p.lsoa, p.new, p.duration, p.type, e.total_floor_area, e.number_habitable_rooms,
    CAST(ROUND((p.price)::numeric,0 ) as integer) as avg_price,
    CAST(ROUND((p.price/e.total_floor_area)::numeric, 3 ) as integer) as sqm_price

from prices as p
join epc as e
on e.brn = p.brn
join geo as g
on p.lsoa = g.lsoa
where e.total_floor_area <> 0 and e.number_habitable_rooms <> 0 and e.number_habitable_rooms is not null and e.total_floor_area is not null
and p.year in (2017, 2018, 2019)
and e.number_habitable_rooms = 3
and (e.total_floor_area > 60 and e.total_floor_area < 69)
and p.type = 'D'
and p.duration = 'F'
and p.new = 'N'
;

""")
rows = cur.fetchall()
result = pd.DataFrame(rows, columns = [desc[0] for desc in cur.description])


In [15]:
result.shape

(137, 9)